In [2]:
import os
import json
import pickle
import requests
import collections
import pandas as pd
import numpy as np
from tqdm import tqdm

In [5]:
with open('data/jeopardy_358974_questions_20190612.pkl', 'rb') as f:
    karl_to_question_id = pickle.load(f).to_dict()['question_id']
with open('data/jeopardy_310326_question_player_pairs_20190612.pkl', 'rb') as f:
    records_df = pickle.load(f)
    
def get_result(card):
    record_id = karl_to_question_id[int(card['question_id'])]
    prob = (records_df[records_df.question_id == record_id]['correct'] / 2 + 0.5).mean()
    return 'correct' if prob > 0.5 else 'wrong'

In [15]:
with open('data/diagnostic_questions.pkl', 'rb') as f:
    cards = pickle.load(f)
    
shown_cards = []
results = []
while len(shown_cards) < 300:
    r = requests.post('http://127.0.0.1:8000/api/karl/schedule', data=json.dumps(cards))
    r = json.loads(r.text)
    show_card = cards[r['order'][0]]
    show_card['label'] = get_result(show_card)
    shown_cards.append(show_card)
    results.append(1 if show_card['label'] == 'correct' else 0)
    print('{: <40}{: <20}{: <20}{: <20}'.format(
        show_card['answer'],
        show_card['category'],
        show_card['label'],
        np.mean(results)
    ))
    print(r['rationale'])
    print()
    r = requests.post('http://127.0.0.1:8000/api/karl/update', data=json.dumps([show_card]))
    cards = [x for x in cards if x['question_id'] != show_card['question_id']]

Greg, Peter & Bobby                     POP CULTURE         wrong               0.0                 
qrep: 0.675
prob: 0.006
category: 0.000
leitner: 0.000
sm2: 0.000
sum: 0.072
user: 0.381
card: 0.375
topic: 2.000

dark side of the Moon                   POP CULTURE         correct             0.5                 
qrep: 0.839
prob: 0.038
category: 0.000
leitner: 0.000
sm2: 0.000
sum: 0.111
user: 0.663
card: 0.625
topic: 14.000

_The Rocky Horror Picture Show_         POP CULTURE         correct             0.6666666666666666  
qrep: 0.889
prob: 0.069
category: 0.000
leitner: 0.000
sm2: 0.000
sum: 0.137
user: 0.684
card: 0.615
topic: 19.000

Wonder Woman                            POP CULTURE         wrong               0.5                 
qrep: 0.921
prob: 0.273
category: 0.000
leitner: 0.000
sm2: 0.000
sum: 0.283
user: 0.648
card: 0.375
topic: 8.000

Canada                                  WORLD GEOGRAPHY     correct             0.6                 
qrep: 0.255
prob: 0.031
category:

nostalgia                               PSYCHOLOGY          wrong               0.625               
qrep: 0.911
prob: 0.303
category: 0.000
leitner: 0.000
sm2: 0.000
sum: 0.303
user: 0.636
card: 0.333
topic: 5.000

equilibrium                             PHYSICS             correct             0.6341463414634146  
qrep: 0.256
prob: 0.019
category: 1.000
leitner: 0.000
sm2: 0.000
sum: 0.339
user: 0.686
card: 0.667
topic: 5.000

Edison effect                           PHYSICS             wrong               0.6190476190476191  
qrep: 0.953
prob: 0.209
category: 0.000
leitner: 0.000
sm2: 0.000
sum: 0.241
user: 0.609
card: 0.400
topic: 18.000

coherent                                PHYSICS             wrong               0.6046511627906976  
qrep: 0.938
prob: 0.244
category: 0.000
leitner: 0.000
sm2: 0.000
sum: 0.265
user: 0.644
card: 0.400
topic: 14.000

gravity                                 PHYSICS             wrong               0.5909090909090909  
qrep: 0.315
prob: 0.361
category:

the Rough Riders                        AMERICAN HISTORY    wrong               0.5769230769230769  
qrep: 0.872
prob: 0.116
category: 0.000
leitner: 0.000
sm2: 0.000
sum: 0.168
user: 0.616
card: 0.500
topic: 3.000

St. Thomas Aquinas                      RELIGION            correct             0.5822784810126582  
qrep: 0.237
prob: 0.012
category: 1.000
leitner: 0.000
sm2: 0.000
sum: 0.332
user: 0.666
card: 0.667
topic: 3.000

_A Midsummer Night's Dream_             SHAKESPEARE         correct             0.5875              
qrep: 0.064
prob: 0.041
category: 1.000
leitner: 0.000
sm2: 0.000
sum: 0.335
user: 0.666
card: 0.625
topic: 3.000

Berlin                                  SCIENTISTS          correct             0.5925925925925926  
qrep: 0.883
prob: 0.001
category: 1.000
leitner: 0.000
sm2: 0.000
sum: 0.389
user: 0.779
card: 0.778
topic: 8.000

Giant Squid                             ANIMALS             correct             0.5975609756097561  
qrep: 0.969
prob: 0.009
category: 1

_Die Meistersinger_ (_von Nürnburg_)    OPERA               correct             0.603448275862069   
qrep: 0.036
prob: 0.124
category: 0.000
leitner: 0.000
sm2: 0.000
sum: 0.090
user: 0.702
card: 0.714
topic: 12.000

Mowgli                                  FICTIONAL CHARACTERScorrect             0.6068376068376068  
qrep: 0.883
prob: 0.006
category: 1.000
leitner: 0.000
sm2: 0.000
sum: 0.393
user: 0.631
card: 0.625
topic: 15.000

Gulliver                                FICTIONAL CHARACTERSwrong               0.6016949152542372  
qrep: 0.965
prob: 0.349
category: 0.000
leitner: 0.000
sm2: 0.000
sum: 0.341
user: 0.599
card: 0.250
topic: 7.000

mushroom                                LITERATURE          correct             0.6050420168067226  
qrep: 0.927
prob: 0.005
category: 1.000
leitner: 0.000
sm2: 0.000
sum: 0.396
user: 0.641
card: 0.636
topic: 16.000

_The Iliad_                             LITERATURE          correct             0.6083333333333333  
qrep: 0.936
prob: 0.160
category

plagiarism                              BOOKS & AUTHORS     wrong               0.5705128205128205  
qrep: 0.865
prob: 0.435
category: 0.000
leitner: 0.000
sm2: 0.000
sum: 0.391
user: 0.768
card: 0.333
topic: 11.000

Ice/Figure Skater (book written by Peggy Fleming)BOOKS & AUTHORS     wrong               0.5668789808917197  
qrep: 0.949
prob: 0.459
category: 0.000
leitner: 0.000
sm2: 0.000
sum: 0.416
user: 0.626
card: 0.167
topic: 2.000

(Candace) Bushnell                      BOOKS & AUTHORS     wrong               0.5632911392405063  
qrep: 0.947
prob: 0.521
category: 0.000
leitner: 0.000
sm2: 0.000
sum: 0.459
user: 0.646
card: 0.125
topic: 3.000

T-Mobile                                BUSINESS & INDUSTRY correct             0.5660377358490566  
qrep: 0.299
prob: 0.003
category: 1.000
leitner: 0.000
sm2: 0.000
sum: 0.332
user: 0.696
card: 0.692
topic: 3.000

Michelangelo                            SCULPTURE           correct             0.56875             
qrep: 0.924
prob: 0.116
c

Rocky Road (The Depression - bad times) FOOD                correct             0.5641025641025641  
qrep: 0.005
prob: 0.026
category: 1.000
leitner: 0.000
sm2: 0.000
sum: 0.319
user: 0.859
card: 0.833
topic: 7.000

arrabbiata                              FOOD                wrong               0.5612244897959183  
qrep: 0.961
prob: 0.694
category: 0.000
leitner: 0.000
sm2: 0.000
sum: 0.582
user: 0.694
card: 0.000
topic: 3.000

_The Bridges of Madison County_         NOVELS              correct             0.5634517766497462  
qrep: 0.570
prob: 0.339
category: 1.000
leitner: 0.000
sm2: 0.000
sum: 0.594
user: 0.744
card: 0.778
topic: 3.000

Linus Pauling                           PEOPLE              correct             0.5656565656565656  
qrep: 0.214
prob: 0.391
category: 1.000
leitner: 0.000
sm2: 0.000
sum: 0.595
user: 0.761
card: 0.800
topic: 3.000

Jane Seymour                            PEOPLE              correct             0.5678391959798995  
qrep: 0.884
prob: 0.506
category: 0

Cardinal Richelieu                      POLITICIANS         correct             0.5683760683760684  
qrep: 0.960
prob: 0.476
category: 1.000
leitner: 0.000
sm2: 0.000
sum: 0.729
user: 0.881
card: 0.929
topic: 12.000

Alexander Karensky                      POLITICIANS         wrong               0.5659574468085107  
qrep: 0.492
prob: 0.828
category: 0.000
leitner: 0.000
sm2: 0.000
sum: 0.629
user: 0.905
card: 0.077
topic: 12.000

Assisi                                  RELIGION            correct             0.5677966101694916  
qrep: 0.195
prob: 0.452
category: 1.000
leitner: 0.000
sm2: 0.000
sum: 0.636
user: 0.955
card: 1.000
topic: 12.000

dyslexia                                PSYCHOLOGY          correct             0.569620253164557   
qrep: 0.078
prob: 0.226
category: 1.000
leitner: 0.000
sm2: 0.000
sum: 0.466
user: 0.977
card: 1.000
topic: 12.000

depression                              PSYCHOLOGY          correct             0.5714285714285714  
qrep: 0.034
prob: 0.113
categor

front                                   WEATHER             correct             0.5845588235294118  
qrep: 0.005
prob: 0.017
category: 1.000
leitner: 0.000
sm2: 0.000
sum: 0.312
user: 0.998
card: 1.000
topic: 7.000

an aneroid barometer                    WEATHER             wrong               0.5824175824175825  
qrep: 0.977
prob: 0.789
category: 0.000
leitner: 0.000
sm2: 0.000
sum: 0.650
user: 0.989
card: 0.200
topic: 10.000

fog                                     WEATHER             wrong               0.5802919708029197  
qrep: 0.938
prob: 0.818
category: 0.000
leitner: 0.000
sm2: 0.000
sum: 0.667
user: 0.818
card: 0.000
topic: 1.000

Linus Yale                              INVENTORS           wrong               0.5781818181818181  
qrep: 0.603
prob: 0.615
category: 1.000
leitner: 0.000
sm2: 0.000
sum: 0.791
user: 1.000
card: 0.385
topic: 10.000

Charles Macintosh                       INVENTORS           wrong               0.5760869565217391  
qrep: 0.249
prob: 0.700
category: